<a href="https://colab.research.google.com/github/Evgeniya371/PRA3024/blob/main/Task_3_(Reflectance_of_the_mulitlayer_structure).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='orange'> Task 3


###Run each cell below one by one, read the annotations



*   Run this cell

*   The first two lines can be commented once you run the cell (spicy and pandas packages will be already imported into the environment)






In [1]:
# This cell imports all important packages

#!pip install scipy
#!pip install pandas
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib

# Functions


Run without changes





In [27]:
# This function computes a single film matrix
# s-polarization

def single_film_matrix_s(n, o_t, lambda_o, phi):
# n - refraction index of the layer
# o_t - optical thickness of the layer (expressed as a fraction of the wavelength of light)
# lambda_o - wavelength of the incident light (in meters)
# phi - propagation angle (in radians)

    k_0 = 2*np.pi/lambda_o              # Wave number: k=2*pi/wavelength
    optical_thickness = o_t * lambda_o    # Optical thickness = n * d = o_t * lambda_o, where d is geometrical distnace

    # Calculation of the matrix elements:
    # Formulas are from: Stenzel, O. Optical Coatings, vol. 54 (2014)
    m_11 = np.around(np.cos(k_0 * optical_thickness * np.cos(phi)), decimals=5)
    m_12 = - 1j / (n * np.cos(phi))* np.around(np.sin(k_0 * optical_thickness * np.cos(phi)), decimals=5)
    m_21 = - 1j * n * np.cos(phi) * np.around(np.sin(k_0 * optical_thickness * np.cos(phi)), decimals=5)
    m_22 = np.around(np.cos(k_0 * optical_thickness *np.cos(phi)), decimals=5)

    # Assign the calculated values to the matrix elements
    matrix=np.empty((2, 2), dtype=object)
    matrix[0, 0] = m_11
    matrix[0, 1] = m_12
    matrix[1, 0] = m_21
    matrix[1, 1] = m_22
    return matrix


# This function computes a single film matrix
# p-polarization

def single_film_matrix_p(n, o_t, lambda_o, phi):
# n - refraction index of the layer
# o_t - optical thickness of the layer (expressed as a fraction of the wavelength of light)
# lambda_o - wavelength of the incident light (in meters)
# phi - propagation angle (in radians)

    k_0 = 2 * np.pi/lambda_o              # Wave number: k=2*pi/wavelength
    optical_thickness = o_t * lambda_o    # Optical thickness = n * d = o_t * lambda_o, where d is geometrical distnace

    # Calculation of the matrix elements:
    # Formulas are from: Stenzel, O. Optical Coatings, vol. 54 (2014)
    m_11 = np.around(np.cos(k_0 * optical_thickness * np.cos(phi)), decimals=5)
    m_12 = - 1j * n / np.cos(phi) * np.around(np.sin( k_0 * optical_thickness * np.cos(phi) ), decimals=5)
    m_21 = - 1j * np.cos(phi) / n * np.around(np.sin( k_0 * optical_thickness * np.cos(phi) ), decimals=5)
    m_22 = np.around(np.cos( k_0 * optical_thickness * np.cos(phi) ), decimals=5)

    # Assign the calculated values to the matrix elements
    matrix = np.empty((2, 2), dtype=object)
    matrix[0, 0] = m_11
    matrix[0, 1] = m_12
    matrix[1, 0] = m_21
    matrix[1, 1] = m_22
    return matrix


# This function computes reflection coefficient
# s-polarization

def reflection_coefficient_s(phi_1, phi_sub, n_1, n_sub, M):
# phi_1 - propagation angle in the incidence medium
# phi_sub - propagation angle in the substrate
# n_1 - refraction index  of the incidence medium
# n_sub - refraction index of the substrate material
# M - coating matrix


    # Matrix elements
    m_11 = M[0, 0]
    m_12 = M[0, 1]
    m_21 = M[1, 0]
    m_22 = M[1, 1]

    # Calculation of the reflection coefficient:
    # Formula is taken from: Stenzel, O. Optical Coatings, vol. 54 (2014)
    r = ((m_11 + m_12 * n_sub * np.cos(phi_sub)) * n_1 * np.cos(phi_1) - (m_21 + m_22 * n_sub * np.cos(phi_sub))) / ((m_11 + m_12 * n_sub * np.cos(phi_sub)) * n_1 * np.cos(phi_1) + m_21 + m_22 * n_sub * np.cos(phi_sub))

    return r


# This function computes reflection coefficient
# p-polarization

def reflection_coefficient_p(phi_1, phi_sub, n_1, n_sub, M):
# phi_1 - propagation angle in the incidence medium
# phi_sub - propagation angle in the substrate
# n_1 - refraction index  of the incidence medium
# n_sub - refraction index of the substrate material
# M - coating matrix


    # Matrix elements
    m_11 = M[0, 0]
    m_12 = M[0, 1]
    m_21 = M[1, 0]
    m_22 = M[1, 1]

    # Calculation of the reflection coefficient:
    # Formula is taken from: Stenzel, O. Optical Coatings, vol. 54 (2014)
    r = ((m_11 + m_12 * np.cos(phi_sub) / n_sub) * np.cos(phi_1) / n_1 - (m_21 + m_22 * np.cos(phi_sub) / n_sub)) / ((m_11 + m_12 * np.cos(phi_sub) / n_sub) * np.cos(phi_1)/n_1 + m_21 + m_22 * np.cos(phi_sub) / n_sub)

    return r

# This function computes transmission coefficient
# s-polarization

def transmission_coefficient_s(phi_1, phi_sub, n_1, n_sub, M):
# phi_1 - propagation angle in the incidence medium
# phi_sub - propagation angle in the substrate
# n_1 - refraction index  of the incidence medium
# n_sub - refraction index of the substrate material
# M - coating matrix


    # Matrix elements
    m_11 = M[0, 0]
    m_12 = M[0, 1]
    m_21 = M[1, 0]
    m_22 = M[1, 1]

    # Calculation of the reflection coefficient:
    # Formula is taken from: Stenzel, O. Optical Coatings, vol. 54 (2014)
    t = (2 * n_1 * np.cos(phi_1)) / ((m_11 + m_12 * n_sub * np.cos(phi_sub)) * n_1 * np.cos(phi_1) + m_21 + m_22 * n_sub * np.cos(phi_sub))

    return t


# This function computes transmission coefficient
# p-polarization

def transmission_coefficient_p(phi_1, phi_sub, n_1, n_sub, M):
# phi_1 - propagation angle in the incidence medium
# phi_sub - propagation angle in the substrate
# n_1 - refraction index  of the incidence medium
# n_sub - refraction index of the substrate material
# M - coating matrix


    # Matrix elements
    m_11 = M[0, 0]
    m_12 = M[0, 1]
    m_21 = M[1, 0]
    m_22 = M[1, 1]

    # Calculation of the reflection coefficient:
    # Formula is taken from: Stenzel, O. Optical Coatings, vol. 54 (2014)
    t = (2 * np.cos(phi_1) / n_1) / ((m_11 + m_12 * np.cos(phi_sub) / n_sub) * np.cos(phi_1)/n_1 + m_21 + m_22 * np.cos(phi_sub) / n_sub)

    return t


# This function computes the M_stack matrix and reflection coefficient of the multilayer coating

def stack(layers_dict, lambda_o, phi_1, n_1, n_sub, polarization):
# layers_dict - dictionary that represent a stack of coating layers
# lambda_o - wavelength of the incident light
# phi_1 - propagation angle in the incidence medium
# phi_sub - propagation angle in the substrate
# n_1 - refraction index  of the incidence medium
# n_sub - refraction index of the substrate material
# polarization - polarization of the light wave ('s' or 'p')

    M_stack = np.identity(2) # Initialization of the M_stack matrix. Identity matrix is chosen as the intial matrix, since multiplication by identity matrix does not change the mutliplied matrix.
    lambda_o = lambda_o * 10 ** (-9)   # Conversion from nanometers to meters
    phi_1 = math.radians(phi_1)        # Conversion to radians

    ang_prev = phi_1   # Initialization of the angle_previous variable - propagation in the first medium
    n_prev = n_1       # Initialization of the n_previous variable - propagation in the first medium

    # Iteration over the supplied layers_dict dictionary
    for k, v in layers_dict.items():

        print(k)
        refractive_index = v['refractive index']
        optical_thickness = v['optical thickness']
        angle = angle_of_refraction(angle_previous = ang_prev, n_previous = n_prev, n_current = refractive_index)
        geometrical_thickness_m = optical_thickness*lambda_o/refractive_index
        geometrical_thickness_nm = geometrical_thickness_m/(10 ** (-9))
        print('Geometrical thickness:', geometrical_thickness_nm, 'nm')

        # If input value is 's', compute s-polarization
        if polarization == 's':
           M = single_film_matrix_s(n = refractive_index, o_t = optical_thickness, lambda_o = lambda_o, phi = angle)
           M_stack = np.matmul(M_stack, M)  # Matric multiplication for the successive layers

        # If input value is 'p', compute p-polarization
        if polarization == 'p':
           M = single_film_matrix_p(n = refractive_index, o_t = optical_thickness, lambda_o = lambda_o, phi = angle)
           M_stack = np.matmul(M_stack, M)  # Matric multiplication for the successive layers

        M_stack = M_stack

        # Update the values of the angle_previous and n_previous variables

        if phi_1 == 0:  # If the incident angle is 0, all propagation angles in the multilayer coating are 0
           ang_prev = 0
        else:
           ang_prev =  np.pi/2 - angle # In the right triangle the sum two acute angles is pi/2

        n_prev = refractive_index

    # Calculate the propagation angle of the final layer - substrate
    phi_sub = angle_of_refraction(angle_previous = ang_prev, n_previous = n_prev, n_current = n_sub)

    # If input value is 's', compute s-polarization
    if polarization == 's':
       r = reflection_coefficient_s(phi_1, phi_sub, n_1, n_sub, M_stack)
       t = transmission_coefficient_s(phi_1, phi_sub, n_1, n_sub, M_stack)

    # If input value is 'p', compute p-polarization
    if polarization == 'p':
       r = reflection_coefficient_p(phi_1, phi_sub, n_1, n_sub, M_stack)
       t = transmission_coefficient_p(phi_1, phi_sub, n_1, n_sub, M_stack)

    # Computation of the characheristic matrix of the stack in the reversed order

    M_reversed_stack = np.empty((2, 2), dtype=object) # Initialisation of the reversed stack matrix

    M_reversed_stack[0, 0] = M_stack[1, 1]
    M_reversed_stack[0, 1] = M_stack[0, 1]
    M_reversed_stack[1, 0] = M_stack[1, 0]
    M_reversed_stack[1, 1] = M_stack[0, 0]

    # If input value is 's', compute s-polarization
    if polarization == 's':
       r_reversed = reflection_coefficient_s(phi_1 = phi_sub, phi_sub = phi_1, n_1 = n_sub, n_sub = n_1, M = M_reversed_stack)
       t_reversed = transmission_coefficient_s(phi_1 = phi_sub, phi_sub = phi_1, n_1 = n_sub, n_sub = n_1, M = M_reversed_stack)

    # If input value is 'p', compute p-polarization
    if polarization == 'p':
       r_reversed = reflection_coefficient_p(phi_1 = phi_sub, phi_sub = phi_1, n_1 = n_sub, n_sub = n_1, M = M_reversed_stack)
       t_reversed = transmission_coefficient_p(phi_1 = phi_sub, phi_sub = phi_1, n_1 = n_sub, n_sub = n_1, M = M_reversed_stack)


    return M_stack, r, t, M_reversed_stack, r_reversed, t_reversed



# This function finds the propagation angle in the current medium when the angle of propagation of the previous medium and
# the refraction indices of the previous/current medium are provided
# The angle is calculated using Snell's law: n_previous * sin(angle_previous) = n_current * sin(angle)

def angle_of_refraction(angle_previous, n_previous, n_current):


    print('The previous propagation angle:', angle_previous, 'rad')

    # Determine the critical angle of refraction: sin(angle_crit) = n_current/n_previous * sin(90) = n_current/n_previous

     # Check this condition: n_previous > n_current
    if n_previous > n_current:
       print('n_previous > n_current: the condition for the total reflection should be checked')

       sin_of_angle_crit = n_current/n_previous      # sin(angle_crit) = n_current/n_previous

       if (sin_of_angle_crit < -1) or (sin_of_angle_crit > 1):
           print('sin(angle_crit) is an invalid value! sin is a bounded function: -1 < sin(angle_crit) < 1 ')
           return None
       angle_crit = np.arcsin(sin_of_angle_crit)     # angle_crit = arcsin(sin(angle_crit)) = arcsin(n/n_previous)
       print('The critical angle:', angle_crit, 'rad')

       # Check the condition for the total internal refelection
       if angle_previous > angle_crit:
           angle_of_refraction  = - angle_previous   # Total internal refelection
           print('Total internal reflection! Angle:', angle_of_refraction, 'rad')
           print('Execution is terminated. No forward propagation through the layers due to the total internal reflection')
           return None

       # If angle_previous is not greater than angle_crit, continue the calculation of the refraction angle:
       sin_of_angle = n_previous * np.sin(angle_previous)/n_current

       if ( sin_of_angle < -1) or ( sin_of_angle > 1):
           print('sin(angle_of_refraction) is an invalid value! sin is a bounded function: -1 < sin(angle_of_refraction) < 1 ')
           return None
       angle_of_refraction  = np.arcsin(sin_of_angle)

       return angle_of_refraction # Output is in radians!

    else:

       sin_of_angle = n_previous * np.sin(angle_previous)/n_current

       if ( sin_of_angle < -1) or ( sin_of_angle > 1):
           print('sin(angle_of_refraction) is an invalid value! sin is bounded function: -1 < sin(angle_of_refraction) < 1 ')
           return None
       angle_of_refraction  = np.arcsin(sin_of_angle)

       print('The propagation angle:', angle_of_refraction, 'rad')

       return angle_of_refraction # Output is in radians!




*   Fill in the empty spaces in this cell. Namely, specify the parameter values and define the multilayer structure:


The multilayer structure should be difined in the following manner:
layers = {



*   Run

**Examples of the implementation are provided in the next cells.**

In [ ]:
# Input values
angle_1 =        # [degrees]   # Angle of incidence (with respect to the normal to the surface). The incidence medium is usually air.
refraction_index_1 =           # Refraction index of the incidence medium (usually air)
refraction_index_sub =         # Refraction index of the substrate material
polarization = ''              # Select the polarization by specifing the input as 's' or 'p'
wavelength =     # [nm]        # Wavelength of the incident light in nanometers [nm]

# Define a stack of coating layers

layers = {

    'layer 1': {
         'thickness': ,           # Thickness of the layer (expressed as a fraction of the wavelength!!!)
         'refractive index': ,    # Refractive index of the layer
         },
    'layer 2': {
         'thickness': ,
         'refractive index': ,
         },
     'layer 3': {
         'thickness': ,
         'refractive index': ,
         }
    'layer 4': {
         'thickness': ,
         'refractive index': ,
         }

    # Add next layers below, preserve the order of layers!
     ### ...........
     ### ...........
     ### ...........
     }

# Compute M_stack matrix and refraction coefficint of the multilayer structure
M_stack, r, t, M_reversed_stack, r_reversed, t_reversed = stack(layers_dict = layers, lambda_o = wavelength, phi_1 = angle_1, n_1 = refraction_index_1, n_sub = refraction_index_sub, polarization = polarization)

# Print the output
print('Matrix, stack:')
print(M_stack)
print('Reflection coefficient: ', r)
print('Transmission coefficient: ', t)
print('Matrix, reversed stack:')
print(M_reversed_stack)
print('Reflection coefficient, reversed stack: ', r_reversed)
print('Transmission coefficient, reversed stack: ', t_reversed)


SyntaxError: ignored

## Example: 3 QW layers

Polarization: s

Structure: incidence medium|LHH|substrate

In [18]:
# Input values
angle_1 = 0       # [degrees]   # Angle of incidence (with respect to the normal to the surface). The incidence medium is usually air.
refraction_index_1 = 1          # Refraction index of the incidence medium (usually air)
refraction_index_sub = 1.2      # Refraction index of the substrate material
polarization = 's'              # Select the polarization by specifing the input as 's' or 'p'
wavelength = 800  # [nm]        # Wavelength of the incident light in nanometers [nm]

#wavelength/4=n*d=n*factor*wavelength

# Define a stack of coating layers

layers = {

    'layer 1': {
         'optical thickness': 0.125,       # Thickness of the layer (expressed as a fraction of the refernece wavelength)
         'refractive index': 2,    # Refractive index of the layer
          },
    'layer 2': {
         'optical thickness': 0.0625,
         'refractive index': 4,
          },
     'layer 3': {
         'optical thickness': 0.0625,
         'refractive index': 4,
         }
     }

# Compute M_stack matrix and refraction coefficint of the multilayer structure
M_stack, r, t, M_reversed_stack, r_reversed, t_reversed = stack(layers_dict = layers, lambda_o = wavelength, phi_1 = angle_1, n_1 = refraction_index_1, n_sub = refraction_index_sub, polarization = polarization)

# Print the output
print('Matrix, stack:')
print(M_stack)
print('Reflection coefficient: ', r)
print('Transmission coefficient: ', t)
print('Matrix, reversed stack:')
print(M_reversed_stack)
print('Reflection coefficient, reversed stack: ', r_reversed)
print('Transmission coefficient, reversed stack: ', t_reversed)





layer 1
The previous propagation angle: 0.0 rad
The propagation angle: 0.0 rad
Geometrical thickness: 50.0 nm
layer 2
The previous propagation angle: 0 rad
The propagation angle: 0.0 rad
Geometrical thickness: 12.5 nm
layer 3
The previous propagation angle: 0 rad
The propagation angle: 0.0 rad
Geometrical thickness: 12.5 nm
The previous propagation angle: 0 rad
n_previous > n_current: the condition for the total reflection should be checked
The critical angle: 0.3046926540153975 rad
Matrix, stack:
[[(-0.499991344416576+0j) -0.375001883323272j]
 [-3.000001666568832j (0.250005722221296+0j)]]
Reflection coefficient:  (-0.7232577463704372-0.2738079590280903j)
Transmission coefficient:  (-0.03349116651056975+0.5777681211353016j)
Matrix, reversed stack:
[[(0.250005722221296+0j) -0.375001883323272j]
 [-3.000001666568832j (-0.499991344416576+0j)]]
Reflection coefficient, reversed stack:  (-0.7500506196659467+0.18840550430043512j)
Transmission coefficient, reversed stack:  (-0.04018939981268369

## Example: 2 QW layers

Polarization: s

Structure: incidence medium|LH|substrate

In [28]:
# Input values
angle_1 = 0      # [degrees]     # Angle of incidence (with respect to the normal to the surface). The incidence medium is usually air.
refraction_index_1 = 1            # Refraction index of the incidence medium (usually air)
refraction_index_sub = 1.45       # Refraction index of the substrate material
polarization = 's'                # Select the polarization by specifing the input as 's' or 'p'
wavelength = 1000  # [nm]         # Wavelength of the incident light in nanometers [nm]


# Define a stack of coating layers

layers = {

    'layer 1': {
         'optical thickness': 0.25,       # Optical thickness of the layer (expressed as a fraction of the refernece wavelength)
         'refractive index': 1.5,         # Refractive index of the layer
          },
    'layer 2': {
         'optical thickness': 0.25,       # Optical thickness of the layer (expressed as a fraction of the refernece wavelength)
         'refractive index': 2,           # Refractive index of the layer
          }

         }


# Compute M_stack matrix and refraction coefficint of the multilayer structure
M_stack, r, t, M_reversed_stack, r_reversed, t_reversed = stack(layers_dict = layers, lambda_o = wavelength, phi_1 = angle_1, n_1 = refraction_index_1, n_sub = refraction_index_sub, polarization = polarization)

# Print the output
print('Matrix, stack:')
print(M_stack)
print('Reflection coefficient: ', r)
print('Transmission coefficient: ', t)
print('Matrix, reversed stack:')
print(M_reversed_stack)
print('Reflection coefficient, reversed stack: ', r_reversed)
print('Transmission coefficient, reversed stack: ', t_reversed)



layer 1
The previous propagation angle: 0.0 rad
The propagation angle: 0.0 rad
Geometrical thickness: 166.66666666666669 nm
layer 2
The previous propagation angle: 0 rad
The propagation angle: 0.0 rad
Geometrical thickness: 125.00000000000001 nm
The previous propagation angle: 0 rad
n_previous > n_current: the condition for the total reflection should be checked
The critical angle: 0.8110343942875815 rad
Matrix, stack:
[[(-1.3333333333333333+0j) 0j]
 [0j (-0.75+0j)]]
Reflection coefficient:  (0.10154905335628228-0j)
Transmission coefficient:  (-0.8261617900172117-0j)
Matrix, reversed stack:
[[(-0.75+0j) 0j]
 [0j (-1.3333333333333333+0j)]]
Reflection coefficient, reversed stack:  (-0.10154905335628228-0j)
Transmission coefficient, reversed stack:  (-1.197934595524957-0j)


## Example: 1 QW layer

Polarization: s

Structure: incidence medium|layer|substrate

In [30]:
# Input values

angle_1 = 0       # [degrees]    # Angle of incidence (with respect to the normal to the surface). The incidence medium is usually air.
refraction_index_1 = 1           # Refraction index of the incidence medium (usually air)
refraction_index_sub = 1.45      # Refraction index of the substrate material
polarization = 's'               # Select the polarization by specifing the input as 's' or 'p'
wavelength = 800  # [nm]         # Wavelength of the incident light in nanometers [nm]


# Define a stack of coating layers
layers = {

    'layer 1': {
         'optical thickness': 0.25,          # Optical thickness of the layer (expressed as a fraction of the refernece wavelength)
         'refractive index': 2,              # Refractive index of the layer
          }
    }

# Compute M_stack matrix and refraction coefficint of the multilayer structure
M_stack, r, t, M_reversed_stack, r_reversed, t_reversed = stack(layers_dict = layers, lambda_o = wavelength, phi_1 = angle_1, n_1 = refraction_index_1, n_sub = refraction_index_sub, polarization = polarization)

# Print the output
print('Matrix, stack:')
print(M_stack)
print('Reflection coefficient: ', r)
print('Transmission coefficient: ', t)
print('Matrix, reversed stack:')
print(M_reversed_stack)
print('Reflection coefficient, reversed stack: ', r_reversed)
print('Transmission coefficient, reversed stack: ', t_reversed)


layer 1
The previous propagation angle: 0.0 rad
The propagation angle: 0.0 rad
Geometrical thickness: 100.0 nm
The previous propagation angle: 0 rad
n_previous > n_current: the condition for the total reflection should be checked
The critical angle: 0.8110343942875815 rad
Matrix, stack:
[[0j -0.5j]
 [-2j 0j]]
Reflection coefficient:  (-0.46788990825688065+0j)
Transmission coefficient:  (-0+0.7339449541284403j)
Matrix, reversed stack:
[[0j -0.5j]
 [-2j 0j]]
Reflection coefficient, reversed stack:  (-0.46788990825688065+0j)
Transmission coefficient, reversed stack:  (-0+1.0642201834862384j)


## Example: 3 QW layers

Polarization: s

Structure: incidence medium|LHL|substrate

In [29]:
# Input values

angle_1 = 0       # [degrees]    # Angle of incidence (with respect to the normal to the surface). The incidence medium is usually air.
refraction_index_1 = 1           # Refraction index of the incidence medium (usually air)
refraction_index_sub = 1.45      # Refraction index of the substrate material
polarization = 's'               # Select the polarization by specifing the input as 's' or 'p'
wavelength = 800  # [nm]         # Wavelength of the incident light in nanometers [nm]


# Define a stack of coating layers
layers = {

      'layer 1': {
         'optical thickness': 0.25,       # Optical thickness of the layer (expressed as a fraction of the refernece wavelength)
         'refractive index': 1.5,         # Refractive index of the layer
          },
     'layer 2': {
         'optical thickness': 0.25,       # Optical thickness of the layer (expressed as a fraction of the refernece wavelength)
         'refractive index': 2,           # Refractive index of the layer
          },
     'layer 3': {
        'optical thickness': 0.25,       # Optical thickness of the layer (expressed as a fraction of the refernece wavelength)
        'refractive index': 1.5,         # Refractive index of the layer
         }
    }

# Compute M_stack matrix and refraction coefficint of the multilayer structure
M_stack, r, t, M_reversed_stack, r_reversed, t_reversed = stack(layers_dict = layers, lambda_o = wavelength, phi_1 = angle_1, n_1 = refraction_index_1, n_sub = refraction_index_sub, polarization = polarization)

# Print the output
print('Matrix, stack:')
print(M_stack)
print('Reflection coefficient: ', r)
print('Transmission coefficient: ', t)
print('Matrix, reversed stack:')
print(M_reversed_stack)
print('Reflection coefficient, reversed stack: ', r_reversed)
print('Transmission coefficient, reversed stack: ', t_reversed)


layer 1
The previous propagation angle: 0.0 rad
The propagation angle: 0.0 rad
Geometrical thickness: 133.33333333333331 nm
layer 2
The previous propagation angle: 0 rad
The propagation angle: 0.0 rad
Geometrical thickness: 100.0 nm
layer 3
The previous propagation angle: 0 rad
n_previous > n_current: the condition for the total reflection should be checked
The critical angle: 0.848062078981481 rad
Geometrical thickness: 133.33333333333331 nm
The previous propagation angle: 0 rad
n_previous > n_current: the condition for the total reflection should be checked
The critical angle: 1.3118747847886754 rad
Matrix, stack:
[[0j 0.8888888888888888j]
 [1.125j 0j]]
Reflection coefficient:  (0.06789413118527036+0j)
Transmission coefficient:  -0.8285385500575374j
Matrix, reversed stack:
[[0j 0.8888888888888888j]
 [1.125j 0j]]
Reflection coefficient, reversed stack:  (0.06789413118527036+0j)
Transmission coefficient, reversed stack:  -1.2013808975834293j


# Example: arbitrary thickness of layers

Polarization: s

Structure: incidence medium|layer_1, layer_2, layer_3, layer_4|substrate

In [ ]:
# Input values
angle_1 = 60      # [degrees]   # Angle of incidence (with respect to the normal to the surface). The incidence medium is usually air.
refraction_index_1 = 1          # Refraction index of the incidence medium (usually air)
refraction_index_sub = 1.2      # Refraction index of the substrate material
polarization = 's'              # Select the polarization by specifing the input as 's' or 'p'
wavelength = 800  # [nm]        # Wavelength of the incident light in nanometers [nm]

# Define a stack of coating layers
# Propagation angle in the layer (in degrees)

layers = {

    'layer 1': {
         'optical thickness': 0.1,         # Optical thickness of the layer (expressed as a fraction of the refernece wavelength)
         'refractive index': 2,    # Refractive index of the layer
          },
    'layer 2': {
         'optical thickness': 0.25,
         'refractive index': 4,
         },
     'layer 3': {
         'optical thickness': 0.2,
         'refractive index': 1.4,
         },
    'layer 4': {
         'optical thickness': 0.4,
         'refractive index': 1.1,
         }
     }

# Compute M_stack matrix and refraction coefficint of the multilayer structure
M_stack, r, t, M_reversed_stack, r_reversed, t_reversed = stack(layers_dict = layers, lambda_o = wavelength, phi_1 = angle_1, n_1 = refraction_index_1, n_sub = refraction_index_sub, polarization = polarization)

# Print the output
print('Matrix, stack:')
print(M_stack)
print('Reflection coefficient: ', r)
print('Transmission coefficient: ', t)
print('Matrix, reversed stack:')
print(M_reversed_stack)
print('Reflection coefficient, reversed stack: ', r_reversed)
print('Transmission coefficient, reversed stack: ', t_reversed)


In [41]:
# Test of the angle_of_refraction() function

#print('Angle of refraction:', angle_of_refraction(angle_previous = 0, n_previous = 1.5, n_current = 1.45), 'rad')  # - normal incidence

#print('Angle of refraction:', angle_of_refraction(angle_previous = np.pi/3, n_previous = 1.5, n_current = 1.45), 'rad')  # - 60 degrees angle, no total internal reflection

print('Angle of refraction:', angle_of_refraction(angle_previous = 8*np.pi/18, n_previous = 1.5, n_current = 1.45), 'rad') # - 80 degrees angle, total internal reflection should be observed


The previous propagation angle: 1.3962634015954636 rad
n_previous > n_current: the condition for the total reflection should be checked
The critical angle: 1.3118747847886754 rad
Total internal reflection! Angle: -1.3962634015954636 rad
Execution is terminated. No forward propagation through the layers due to the total internal reflection
Angle of refraction: None rad


In [ ]:
def Transmittance_and_Reflectance(r, t, r_reversed, t_reversed, d_sub, n_sub, n_1, phi_1, frequency):


